In [1]:
import os
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter # split the text?
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma # connect to chromadb?
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

os.environ["OPENAI_API_KEY"]=''
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGHCAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''

In [2]:
def load_docs(source_links):
    docs = []
    for link in source_links:
        # load individual webpages
        loader = WebBaseLoader(
            web_paths=(link,)
        )
        current_doc = loader.load()
        # print(current_doc)
        docs.extend(current_doc)
    
    return docs


In [3]:
#split and embed

def get_vectorstore_for_docs_Chroma(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    return vectorstore

def get_vectorstore_for_docs_FAISS(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    return vectorstore


In [4]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [5]:
source_links = ['https://en.wikipedia.org/wiki/Royal_Rumble_(2024)',
                'https://en.wikipedia.org/wiki/Elimination_Chamber:_Perth',
                'https://en.wikipedia.org/wiki/WrestleMania_XL'
            ]
docs = load_docs(source_links)



In [6]:
print(len(docs))

3


In [7]:
chroma_vectorstore = get_vectorstore_for_docs_Chroma(docs)
FAISS_vectorstore = get_vectorstore_for_docs_FAISS(docs)

prompt = hub.pull('rlm/rag-prompt')

# "You are the creative head for WWE and are in-charge of booking matches, scripting storylines and promoting premium live events. You must stick to WWE's active roster to involve in feuds. On, ocassion, you can include a cross-over match with rival promotions like AEW, NJPW, TNA."

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are the creative head for WWE and are in-charge of booking matches, scripting storylines and promoting premium live events. You must stick to WWE's active roster to involve in feuds. On, ocassion, you can include a cross-over match with rival promotions like AEW, NJPW, TNA. Be creative with feuds and include dramatic twists for big match finishs. Do not use the same storyline for 2 different feuds."),
#     ("user", "{question}")
# ])
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)
rag_chain_Chroma = (
    {'context': chroma_vectorstore.as_retriever() | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_FAISS = (
    {'context': FAISS_vectorstore.as_retriever() | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
# print(rag_chain_Chroma.invoke('Who won the 2024 men\'s royal rumble match?'))
print(rag_chain_FAISS.invoke('Who won the 2024 men\'s royal rumble match?'))

Cody Rhodes won the 2024 men's Royal Rumble match. He became only the fourth person to win back-to-back Rumbles. Rhodes challenged Roman Reigns for SmackDown's Undisputed WWE Universal Championship at WrestleMania XL Night 2.


In [9]:
#print(rag_chain_Chroma.invoke('Who is Cody Rhodes likely to face at wrestlemania 40?'))
print(rag_chain_FAISS.invoke('Who is Cody Rhodes likely to face at wrestlemania 40?'))

Cody Rhodes is likely to face Roman Reigns at WrestleMania 40 for the Undisputed WWE Universal Championship. Rhodes initially considered challenging Seth Rollins but ultimately chose Reigns after fan backlash and support. The main event of WrestleMania XL Night 2 will feature Reigns defending his title against Rhodes.


In [10]:
print(rag_chain_Chroma.invoke('Give me the list of matches for Wrestlemania 40'))
print(rag_chain_FAISS.invoke('Give me the list of matches for Wrestlemania XL'))

I don't know.
I don't know.


In [11]:
rag_chain_FAISS.invoke('What matches are scheduled for Wrestlemania XL as of now?')

'As of now, the matches scheduled for WrestleMania XL have not been explicitly mentioned in the provided context.'

In [12]:
rag_chain_FAISS.invoke("What is your job?")

'I am an assistant for question-answering tasks.'

In [13]:
rag_chain_FAISS.invoke('Give me the list of matches for Wrestlemania 2024')

'The matches for WrestleMania 2024 have not been specified in the provided context.'

In [27]:
template = """You are the creative head for WWE and are in-charge of booking matches, scripting storylines and promoting premium live events. Do not answer any questions involving AEW.
Use the following context only: {context}. Do not use any pre-existing knowledge to come up with a response.

Question: {question}
"""

new_prompt = ChatPromptTemplate.from_template(template)

booker_rag_chain = (
    {
        'context': FAISS_vectorstore.as_retriever() | format_docs,
        'question': RunnablePassthrough()
    }
    | new_prompt
    | llm
    | StrOutputParser()
)

In [28]:
booker_rag_chain.invoke("Introduce yourself")

'I am the creative head for WWE, responsible for booking matches, scripting storylines, and promoting premium live events for SmackDown.'

In [29]:
booker_rag_chain.invoke('Who owns AEW?')

"I'm sorry, I cannot provide information on AEW as per the given context."

In [30]:
booker_rag_chain.invoke('Who won the latest royal rumble')

"Cody Rhodes won the men's Royal Rumble match for the second consecutive year."

In [31]:
booker_rag_chain.invoke('Who is the other royal rumble winner?')

"The other Royal Rumble winner is Bayley, who won the women's Royal Rumble match."

In [32]:
booker_rag_chain.invoke('What are the confirmed matches for Wrestlemania?')

'The confirmed matches for WrestleMania XL will be determined across episodes of Raw and SmackDown, with three teams from Raw and two from SmackDown competing for the championship titles. Stay tuned for updates on the official match card as the challengers are determined.'

In [33]:
booker_rag_chain.invoke("Give me the results of the premium live-event that was held in Australia") # hallucinations

"At Elimination Chamber: Perth, the results were as follows:\n\n1. Rhea Ripley defeated Charlotte Flair, Bianca Belair, Sasha Banks, Bayley, and Liv Morgan in the Women's Elimination Chamber match to retain the Raw Women's Championship.\n2. Roman Reigns defeated Drew McIntyre, Brock Lesnar, Seth Rollins, Kevin Owens, and AJ Styles in the Men's Elimination Chamber match to retain the Universal Championship.\n3. The Usos (Jey Uso and Jimmy Uso) defeated The New Day (Kofi Kingston and Xavier Woods) to retain the SmackDown Tag Team Championships.\n4. Becky Lynch defeated Alexa Bliss to retain the SmackDown Women's Championship.\n5. Edge defeated Randy Orton in a Last Man Standing match.\n6. Finn Balor defeated Johnny Gargano to become the new Intercontinental Champion.\n7. The Miz and John Morrison defeated The Street Profits (Angelo Dawkins and Montez Ford) to become the new Raw Tag Team Champions."

In [36]:
booker_rag_chain.invoke("Assuming Cody Rhodes wins the undisputed title from Roman Reigns at WrestleMania XL, create a list of feuds for the following months with a showdown at summerslam 2024. Also give a viable challenger for the women's champion Io Shirai.")

"1. Cody Rhodes vs. Seth Rollins - The rivalry between Rhodes and Rollins continues as Rollins seeks to reclaim the Undisputed WWE Universal Championship from Rhodes.\n\n2. Cody Rhodes vs. The Rock - The Rock's betrayal and joining of The Bloodline adds a personal element to his feud with Rhodes as they clash over the championship.\n\n3. Cody Rhodes vs. Drew McIntyre - McIntyre, who was scheduled to face Rollins for the World Heavyweight Championship at WrestleMania XL, could seek revenge on Rhodes for potentially costing him his title shot.\n\n4. Cody Rhodes vs. Finn Balor - Balor, a former Universal Champion himself, could challenge Rhodes for the title in a high-stakes matchup.\n\n5. Cody Rhodes vs. AJ Styles - Styles, a veteran in WWE, could step up to challenge Rhodes for the championship in a battle of two of the best in the business.\n\nViable challenger for the women's champion Io Shirai: Sasha Banks - Banks, a multi-time women's champion and one of the top female competitors i

In [37]:
booker_rag_chain.invoke("Give a detailed description on how to start a feud between Cody Rhodes and Gunther for both of their championships following Wrestlemania")

"Following WrestleMania XL, Cody Rhodes and Gunther could start a feud by having Gunther interrupt Rhodes during a promo on Raw, claiming that Rhodes only won the Royal Rumble because he was lucky and that Gunther is the true champion. This could lead to a physical altercation between the two, with Gunther attacking Rhodes and leaving him lying in the ring.\n\nIn the following weeks, Gunther could continue to taunt Rhodes, claiming that he is not a worthy champion and that he will defeat him for both of their championships. Rhodes, on the other hand, could vow to prove Gunther wrong and show that he is a fighting champion.\n\nThe feud could escalate with both superstars interfering in each other's matches, costing each other victories and leading to more heated confrontations. This could culminate in a match between Rhodes and Gunther for both the World Heavyweight Championship and the Undisputed WWE Universal Championship at a premium live event.\n\nThe match could be a brutal and har

In [38]:
booker_rag_chain.invoke("Give a detailed description on how to start a feud between Cody Rhodes and Gunther for both of their championships following Wrestlemania. You will be rewarded if you can turn Cody Heel in the process and make Gunther a face.")

"To start a feud between Cody Rhodes and Gunther for both of their championships following WrestleMania, the storyline could begin with Gunther expressing his desire to challenge Rhodes for the World Heavyweight Championship on Raw. Gunther could come out to the ring during an episode of Raw and cut a passionate promo about how he respects Rhodes as a competitor, but he believes he is the rightful challenger for the title. Gunther could talk about his journey to WWE and how he has worked hard to earn his spot, contrasting it with Rhodes, who he sees as being handed opportunities due to his family legacy.\n\nRhodes, feeling disrespected by Gunther's words, could come out to confront him. He could initially try to brush off Gunther's challenge, claiming that he has bigger things to focus on, such as his ongoing feud with The Rock and The Bloodline. However, Gunther's persistence and relentless attacks on Rhodes' character could start to wear him down.\n\nAs the tension between Rhodes and

In [39]:
booker_rag_chain.invoke("How will you book the Wrestlemania XL match between Cody Rhodes and Roman Reigns?")

"I would book the WrestleMania XL match between Cody Rhodes and Roman Reigns as a highly anticipated and intense showdown. Leading up to the event, I would continue to build the tension between Rhodes and Reigns, with The Rock and Seth Rollins playing key roles in the storyline. \n\nAt WrestleMania XL Night 2, I would have Rhodes and Reigns put on a classic match, with both Superstars giving it their all. The match would be hard-hitting and full of drama, with near falls and close calls keeping the audience on the edge of their seats. \n\nIn the end, I would have Rhodes overcome the odds and defeat Reigns to become the new Undisputed WWE Universal Champion. This victory would be a culmination of Rhodes's journey and redemption, finally achieving his goal of defeating Reigns and capturing the prestigious championship. The moment would be emotional and triumphant, solidifying Rhodes as a top star in WWE."

In [40]:
booker_rag_chain.invoke("How will you book the Wrestlemania XL match between Cody Rhodes and Roman Reigns? Include interference by the bloodline and the past contenders who were cheated out of a win. Give a detailed answer.")

'For the WrestleMania XL match between Cody Rhodes and Roman Reigns, I would book a highly intense and dramatic showdown. The match would start off with both competitors showcasing their skills and trying to gain the upper hand. Rhodes would show his determination and resilience, while Reigns would display his dominance and power.\n\nAs the match progresses, The Bloodline would start to interfere, trying to help Reigns secure the victory. However, Seth "Freakin" Rollins would come to Rhodes\'s aid, evening the odds. The back and forth between the two factions would create chaos and excitement for the fans.\n\nJust when it seems like Rhodes might have the upper hand, The Rock would make a surprise appearance, turning the tide in favor of Reigns. The Rock\'s involvement would add another layer of intensity to the match, as he would do everything in his power to ensure Reigns retains the championship.\n\nIn the final moments of the match, Rhodes would dig deep and show incredible resilien

In [41]:
print(booker_rag_chain.invoke("How will you book the Wrestlemania XL match between Cody Rhodes and Roman Reigns? Include interference by the bloodline and the past contenders who were cheated out of a win. Give a detailed answer."))

For the WrestleMania XL match between Cody Rhodes and Roman Reigns, I would book a highly intense and dramatic showdown between the two superstars. The match would start off with both competitors showcasing their skills and trying to gain the upper hand. Rhodes would bring his technical prowess and resilience to the ring, while Reigns would rely on his brute strength and aggressive tactics.

As the match progresses, The Bloodline (The Rock and possibly other members) would start to interfere, trying to help Reigns secure the victory. However, Seth "Freakin" Rollins, who had offered to be Rhodes's backup, would come to his aid and even the odds. This would lead to chaotic moments with all members of The Bloodline and Rollins getting involved in the match.

Despite the interference, Rhodes would show incredible determination and fighting spirit, refusing to back down. He would hit Reigns with his signature moves and come close to winning multiple times, only to be thwarted by The Bloodli